In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('titanic.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [17]:
df = df[['Pclass','Sex','Age','Fare','Survived']].dropna()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    714 non-null    int64  
 1   Sex       714 non-null    object 
 2   Age       714 non-null    float64
 3   Fare      714 non-null    float64
 4   Survived  714 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 33.5+ KB


In [20]:
df['Sex']

,Sex
0,male
1,female
2,female
3,female
4,male
...,...
885,female
886,male
887,female
889,male


In [11]:
df['Sex'] = df['Sex'] =='male'

In [14]:
X =df[['Pclass','Sex','Age','Fare']].values
y = df['Survived'].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
#scaling
scaler = StandardScaler()
X_train , X_test = scaler.fit_transform(X_train), scaler.transform(X_test)

In [15]:
X_train_tensor  =torch.tensor(X_train, dtype =torch.float32)
X_test_tensor =torch.tensor(X_test, dtype =torch.float32)
y_train_tensor =torch.tensor(y_train, dtype =torch.float32).view(-1,1)
y_test_tensor =torch.tensor(y_test, dtype =torch.float32).view(-1,1)

In [36]:
class Classifier (nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 =nn.Linear(4, 16)
    self.relu1 = nn.ReLU()
    self.dropout = nn.Dropout(p=0.3) #default p=0.5
    self.fc2 = nn.Linear(16,8)
    self.tanh1 = nn.Tanh()
    self.fc3 = nn.Linear(8,1)

  def forward (self, x):
    x= self.fc1(x)
    x = self.relu1(x)
    x = self.dropout(x) #!!
    x = self.fc2(x)
    x =self.tanh1(x)
    x = self.fc3(x)
    return x

model = Classifier()

#losss
criterion = nn.BCEWithLogitsLoss()
#optim
optimizer = optim.Adam(model.parameters(), lr =0.01)
#loop
epochs =1000
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()

  #>חזית
  outputs = model(X_train_tensor)
  loss = criterion(outputs, y_train_tensor)

  #עדכון
  loss.backward()
  optimizer.step()

  # הדפסה
  if epoch%100 ==0:
    predictions =  (torch.sigmoid(outputs)>0.5).float()
    accuracy = (predictions == y_train_tensor).float().mean()
    print(f"Epoch {epoch} , Loss: {loss.item()}, Accuracy {accuracy.item()}")
#evaluation
model.eval()
with torch.no_grad():
  predictions = (torch.sigmoid(model(X_test_tensor))>0.5).float()
  accuracy = (predictions ==  y_test_tensor).float().mean().item()
  print (f"Final Accuracy is {accuracy}")

Epoch 0 , Loss: 0.6804068088531494, Accuracy 0.5831874012947083
Epoch 100 , Loss: 0.40707042813301086, Accuracy 0.8248686790466309
Epoch 200 , Loss: 0.39752084016799927, Accuracy 0.821366012096405
Epoch 300 , Loss: 0.3900787830352783, Accuracy 0.821366012096405
Epoch 400 , Loss: 0.38572055101394653, Accuracy 0.8336251974105835
Epoch 500 , Loss: 0.3828713893890381, Accuracy 0.8283712863922119
Epoch 600 , Loss: 0.37000903487205505, Accuracy 0.8476357460021973
Epoch 700 , Loss: 0.36950618028640747, Accuracy 0.8353765606880188
Epoch 800 , Loss: 0.362749308347702, Accuracy 0.8371278643608093
Epoch 900 , Loss: 0.37362799048423767, Accuracy 0.831873893737793
Final Accuracy is 0.8181818127632141
